In [125]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 964926347732827968
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3225408314893130053
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13804827996640473845
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12067914164712434704
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [0]:
import pandas as pd

In [0]:
#loading the training data
train_df=pd.read_csv('/content/train.csv',encoding='utf-8')

In [0]:
#making 2 lists of tweets and correspong sentiments
tweets=list(train_df['Tweet'])
sentiments=list(train_df['Sentiment'])
sentiments

In [0]:
from nltk import FreqDist
fd=dict(FreqDist(sentiments))
fd

{'negative': 4456, 'neutral': 5631, 'positive': 5030}

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
senti=sentiments.copy()

In [0]:
#converting the string labels to integer representations
enc=LabelEncoder()
enc.fit(sentiments)
sentiments_new=enc.transform(sentiments)
sentiments_new

array([0, 2, 2, ..., 0, 0, 0])

In [0]:
tweets

In [0]:
import keras

Using TensorFlow backend.


In [0]:
tokenizer=keras.preprocessing.text.Tokenizer(filters='',lower=False)

In [0]:
# Creates a word to index mapping of every word
tokenizer.fit_on_texts(tweets)

In [0]:
#converts the tweets to sequences of integer representations
tweets = tokenizer.texts_to_sequences(tweets )

In [0]:
# padding all tweets to length of 60
from keras.preprocessing.sequence import pad_sequences
tweets=pad_sequences(tweets,maxlen=60,padding='post')

In [0]:
tweets

In [0]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

53817

In [0]:
len(tweets)

15117

In [0]:
from numpy import array
from keras.utils import to_categorical
tweets = array(tweets)
    

In [0]:
tweets.shape

(15117, 60)

In [0]:
X=tweets
y=sentiments_new

In [0]:
#assigning weights to classes
from sklearn.utils import compute_class_weight
import numpy as np
classWeight = compute_class_weight('balanced', np.unique(y), y) 
classWeight = dict(enumerate(classWeight))

In [0]:
y

In [0]:
seq_length = X.shape[1] 

In [137]:
from keras.models import Sequential, Model
from keras.layers import Dense, BatchNormalization
from keras.layers import LSTM, Bidirectional,CuDNNLSTM,CuDNNGRU,Embedding

# define model
model = Sequential()
model.add(Embedding(vocab_size, 50,input_length=seq_length,trainable=True)) 
model.add((CuDNNGRU(60)))
model.add(BatchNormalization())
#model.add((CuDNNLSTM(100)))
#model.add(Dense(100, activation='relu'))
model.add(Dense(3, activation='softmax'))
print(model.summary())

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 60, 50)            2690850   
_________________________________________________________________
cu_dnngru_14 (CuDNNGRU)      (None, 60)                20160     
_________________________________________________________________
batch_normalization_16 (Batc (None, 60)                240       
_________________________________________________________________
dense_25 (Dense)             (None, 3)                 183       
Total params: 2,711,433
Trainable params: 2,711,313
Non-trainable params: 120
_________________________________________________________________
None


In [ ]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.00001)
# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0001), metrics=['accuracy']) #categorical crossentropy as there are multiple classes of words to predict
# fit model
model_history=model.fit(X, y, batch_size=10, epochs=20,class_weight=classWeight)

In [0]:
#load test data
test=pd.read_csv('/content/test.csv',encoding='utf-8')
test.Tweet=test.Tweet.astype(str)

In [0]:
#lists of test tweets and labels
test_tweets=list(test['Tweet'])
test_y=list(test['Sentiment'])
test_tweets

In [0]:
#convert labels to integers
test_new=enc.transform(test_y)

In [0]:
test_tweets

In [0]:
#convert tweets to sequences of integers
test_tweets= tokenizer.texts_to_sequences(test_tweets )

In [0]:
#pad tweets to length 60
X_test=pad_sequences(test_tweets,maxlen=60,padding='post')

In [0]:
y_test=test_new
np.unique(y_test)

array([0, 1, 2])

In [139]:
#test the model
model.evaluate(X_test,y_test,batch_size=10)

1869/1869 [==============================] - 2s 997us/step


[2.4791621082607374, 0.513108619813169]

In [0]:
#getting predictions to get recall, etc
y_pred=model.predict_classes(X_test)

In [0]:
y_pred

array([2, 2, 0, ..., 0, 0, 0])

In [0]:
enc.inverse_transform([0,1,2])

array(['negative', 'neutral', 'positive'], dtype='<U8')

In [141]:
#getting F1 Score, Recall, precision
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.53      0.53      0.53       533
           1       0.47      0.52      0.50       754
           2       0.56      0.49      0.52       582

    accuracy                           0.51      1869
   macro avg       0.52      0.51      0.52      1869
weighted avg       0.52      0.51      0.51      1869

